In [ ]:
import pandas as pd
import json 
import glob
import faiss
from fuzzywuzzy import fuzz
from llmsherpa.readers import LayoutPDFReader
import matplotlib.pyplot as plt
import numpy as np
import os
import datetime
import torch
from transformers import AutoTokenizer, AutoModel

In [ ]:
import tqdm

In [ ]:
from src.rechunker import Rechunker
from src.encoder.sentence_transformer import Encoder
from src.faiss.flat_idx import FlatIdx
from utils.utils import flatten_list, write_list_to_file, read_list_from_file
from src.eval import Eval


## Data

In [ ]:
save_path = r"C:\Users\J C SINGLA\Downloads\External - take_home_challenge_(withJSONs)\take_home_challenge_(withJSONs)\data"

In [ ]:
all_data_sherpa = read_list_from_file(save_path, "sherpa_paras_and_tables")
filenames_sherpa = read_list_from_file(save_path, "sherpa_paras_and_tables_filenames")
assert (len(all_data_sherpa)==len(filenames_sherpa))

In [ ]:
ground_truth_path = r"C:\Users\J C SINGLA\Downloads\External - take_home_challenge_(withJSONs)\take_home_challenge_(withJSONs)\document_questions.xlsx"
ground_truth = pd.read_excel(ground_truth_path)
ground_truth_text = ground_truth
test_data = list(ground_truth_text["relevant questions"])
test_labels = list(ground_truth_text["answer"])

In [ ]:
len(test_labels), len(all_data_sherpa)

## Model

In [ ]:
def normalize_vectors(vectors):
    """Normalize vectors to unit length."""
    norm = np.linalg.norm(vectors, axis=1, keepdims=True)
    return vectors / np.where(norm == 0, 1, norm)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('facebook/dragon-plus-query-encoder')
query_encoder = AutoModel.from_pretrained('facebook/dragon-plus-query-encoder')
context_encoder = AutoModel.from_pretrained('facebook/dragon-plus-context-encoder')

In [ ]:
query =  test_data
contexts = all_data_sherpa

In [ ]:
ctx_emb = torch.empty((0, 768))
for i in tqdm.tqdm(range(len(all_data_sherpa))):
    ctx_input = tokenizer(contexts[i:i+1], padding=True, truncation=True, return_tensors='pt', max_length = 512)
    temp_emb = context_encoder(**ctx_input).last_hidden_state[:, 0, :]
    ctx_emb = torch.cat((ctx_emb, temp_emb), dim=0)

In [ ]:
ctx_emb_ = normalize_vectors(ctx_emb.detach().numpy())

In [ ]:
save_path = r"C:\Users\J C SINGLA\Downloads\External - take_home_challenge_(withJSONs)\take_home_challenge_(withJSONs)\data\tensors\dragon_paras_norm.pt"
torch.save(ctx_emb_, save_path)

QUERY VECTORS

In [ ]:
xq = torch.empty((0, 768))
for i in tqdm.tqdm(range(len(test_data))):
    q_input = tokenizer(test_data[i:i+1], padding=True, truncation=True, return_tensors='pt', max_length = 512)
    temp_emb = context_encoder(**q_input).last_hidden_state[:, 0, :]
    xq = torch.cat((xq, temp_emb), dim=0)

In [ ]:
xq_ = normalize_vectors(xq.detach().numpy())

In [ ]:
index = faiss.IndexFlatL2(ctx_emb.shape[1])

In [ ]:
index.add(ctx_emb_)

## Search

In [ ]:
k = 10
distances, indices = index.search(xq_, k)

In [ ]:
ret_context = []
for i in range(len(test_data)):
    retrieved_items = [contexts[i] for i in list(indices[i])]
    ret_context.append(retrieved_items)

In [ ]:
metric = Eval(k=10)
recall, incorrect, correct = metric.recall_k(test_labels, ret_context)

In [ ]:
print ("Recall is ", recall)
print ("MRR is ", metric.mean_reciprocal_rank(ret_context, test_labels))

In [ ]:
incorrect[3]

In [ ]:
test_data[3]